In [2]:
import pandas as pd

**Creación de lista_artistas_id.csv**

A partir del csv lista_artistas_spotify, que contiene una lista de solo nombres de artistas (únicos), vamos a crear un csv que nos facilite la relaciones y consultas en SQL, ya que contendrá un id que será el punto de unión con las tablas que crearemos con los datos de extraídos de Spotify, MusicBraniz y LastFm 

Estos datos serán los que contendrá la tabla **artistas_spotify** en SQL

In [3]:
# Leemos el csv con pandas
lista_artistas_id = pd.read_csv('lista_artistas_spotify.csv')

In [4]:
# #### Esto no hace falta###### la lista ya está con artistas únicos
lista_artistas_id = lista_artistas_id.drop_duplicates()

In [ ]:
# Creamos un id_artista, que será único para cada nombre de artista.
# Para ello samos un range 
lista_artistas_id['id_artista'] = range(1, len(lista_artistas_id) + 1)

In [6]:
# Renombrarmos para que la columna que contiene el nombre de artistas, tenga cabecera
lista_artistas_id = lista_artistas_id.rename(columns={'0': 'artista'})

In [7]:
lista_artistas_id

,artista,id_artista
0,Tory Lanez,1
1,PinkPantheress,2
2,Eyedress,3
3,Jordana,4
4,Tate McRae,5
...,...,...
1885,Quando Rondo,1886
1886,Mighty Bay,1887
1887,Vin Jay,1888
1888,Fat Nick,1889


In [ ]:
# Y la convertimos en csv para poder insertar los datos en SQL
# Con esto ya tenemos una lista de nombres de artistas y cada uno tiene un id único
lista_artistas_id.to_csv('lista_artistas_id.csv', index=False)

**Creación de spotify_id.csv**

A partir del csv spotify_df_concatenado,vamos a crear otro, que, a parte de los datos resultantes de la extracción y limpieza de los extraídos en la API de Spotify, contendrá un id que identificará al artista y que se corresponderá con el que ya hemos creado en lista_artistas_id.csv. 
Esto nos permitirá relacionar las tablas que crearemos para ambos en SQL 

Estos datos serán los que contendrá la tabla **canciones_spotify** en SQL

In [9]:
# Leemos el archivo csv
spotify = pd.read_csv('spotify_df_concatenado.csv')

In [11]:
spotify

,artista,tipo,título_canción,nombre_album,año,genre
0,Tory Lanez,track,The Color Violet,Alone At Prom,2021,pop
1,PinkPantheress,track,Pain,to hell with it,2021,pop
2,Eyedress,track,Something About You,Mulholland Drive,2021,pop
3,Jordana,track,Better in the Dark,Summer's Over,2021,pop
4,Tate McRae,track,you broke me first,TOO YOUNG TO BE SAD,2021,pop
...,...,...,...,...,...,...
7995,Destroy Lonely,track,LOUVRE,NS+ (ULTRA),2022,rap
7996,Future,track,GOLD STACKS,I NEVER LIKED YOU,2022,rap
7997,XXXTENTACION,track,rare,LOOK AT ME: THE ALBUM,2022,rap
7998,SadBoyProlific,track,Merlin,Merlin,2022,rap


In [ ]:
# Vamos a hacer que spotify herede el id de artista lista_artistas_id y, para ello,
# unimos los dos DataFrames en base a la columna 'artista', manteniendo el 
# mismo número de registros de spotify
spotify_id = pd.merge(lista_artistas_id, spotify, on='artista', how='left')

spotify_id


,artista,id_artista,tipo,título_canción,nombre_album,año,genre
0,Tory Lanez,1,track,The Color Violet,Alone At Prom,2021,pop
1,Tory Lanez,1,track,Lavender Sunflower,Alone At Prom,2021,pop
2,Tory Lanez,1,track,Lady Of Namek,Alone At Prom,2021,pop
3,Tory Lanez,1,track,And This is Just The Intro,PLAYBOY,2021,pop
4,Tory Lanez,1,track,Y.D.L.R.,PLAYBOY,2021,pop
...,...,...,...,...,...,...,...
7995,Quando Rondo,1886,track,Want Me Dead,3860,2022,rap
7996,Mighty Bay,1887,track,GOOD NEWS (Intro),BLOND BOY NEWS,2022,rap
7997,Vin Jay,1888,track,Drop,Drop,2022,rap
7998,Fat Nick,1889,track,Mom…It's Not a Phase (feat. Kxllswxtch),Hope You're Proud,2022,rap


In [ ]:
# la columna de artista ya no nos sirve, porque ese dato lo va a suministrar la tabla
# artistas_spotify, así que lo borramos
spotify_id = spotify_id.drop(columns=['artista'])

spotify_id

,id_artista,tipo,título_canción,nombre_album,año,genre
0,1,track,The Color Violet,Alone At Prom,2021,pop
1,1,track,Lavender Sunflower,Alone At Prom,2021,pop
2,1,track,Lady Of Namek,Alone At Prom,2021,pop
3,1,track,And This is Just The Intro,PLAYBOY,2021,pop
4,1,track,Y.D.L.R.,PLAYBOY,2021,pop
...,...,...,...,...,...,...
7995,1886,track,Want Me Dead,3860,2022,rap
7996,1887,track,GOOD NEWS (Intro),BLOND BOY NEWS,2022,rap
7997,1888,track,Drop,Drop,2022,rap
7998,1889,track,Mom…It's Not a Phase (feat. Kxllswxtch),Hope You're Proud,2022,rap


In [ ]:
# Convertimos el archivo en el csv que usaremos para la insercción de datos en SQL
spotify_id.to_csv('spotify_id.csv', index=False)

**Creación de musicbrainz_id.csv**

A partir del csv musicbrainz_df_concatenado, hacemos el mismo procedimiento que en el anterior, para obtener los datos extraídos en la API Music Brainz, que van a incluir ese id que identificará al artista y que se corresponderá con el que ya hemos creado en lista_artistas_id.csv, y nos permita la relación entre las tablas en SQL.

Estos datos serán los que contendrá la tabla **info_artistas_musicbrainz** en SQL

In [13]:
# Leemos el csv que tiene los datos extraidos
musicbrainz_id = pd.read_csv('musicbrainz_df_concatenado.csv')

In [ ]:
# Renombramos las cabeceres del Dataframe
musicbrainz_id = musicbrainz_id.rename(columns={'Nombre': 'artista'})
# Y eliminamos la primera columna, que contiene un índice que crea el DataFrame
musicbrainz_id = musicbrainz_id.drop(columns=['Unnamed: 0'])

In [ ]:
# Unimos la lista lista_artistas_id y la de musicbrainz_id para conseguir el id de artista
musicbrainz_id = pd.merge(lista_artistas_id, musicbrainz_id, on='artista', how='left')

In [89]:
musicbrainz_id = musicbrainz_id.drop_duplicates()

In [ ]:
# la columna de artista ya no nos sirve, porque ese dato lo va a suministrar la tabla
# artistas_spotify, así que lo borramos
musicbrainz_id = musicbrainz_id.drop(columns=['artista'])

In [ ]:
# Convertimos el archivo en el csv que usaremos para la insercción de datos en SQL
musicbrainz_id.to_csv('musicbrainz_id.csv', index=False)

In [92]:
musicbrainz_id

,id_artista,Tipo,País de Origen,Área de Origen,Inicio de actividad,Fin de actividad,Fecha nacimiento,Fecha fallecimiento
0,1,Person,CA,Brampton,-,-,1992-07-27,NaN
1,2,Person,NaN,Bath,-,-,2001-04-19,NaN
2,3,Person,PH,Manila,-,-,1990-05-28,NaN
3,4,Person,NaN,"Washington, D.C.",-,-,2000-06-22,NaN
4,5,Person,CA,Calgary,-,-,2003-07-01,NaN
...,...,...,...,...,...,...,...,...
1891,1886,Person,NaN,Savannah,-,-,1999,NaN
1892,1887,NaN,NaN,NaN,-,-,NaN,NaN
1893,1888,NaN,NaN,NaN,-,-,NaN,NaN
1894,1889,Person,US,NaN,-,-,1994-09-06,NaN


**Creación de lastfm_id.csv**

A partir del csv lastfm_df_concatenado, hacemos el mismo procedimiento que en el anterior, para obtener los datos extraídos en la API Last FM, que van a incluir ese id que identificará al artista y que se corresponderá con el que ya hemos creado en lista_artistas_id.csv, y nos permita la relación entre las tablas en SQL.

Estos datos serán los que contendrá la tabla **info_artistas_lastfm** en SQL

In [ ]:
# Leemos el csv que contiene los datos
lastfm_id = pd.read_csv('lastfm_df_concatenado.csv')

In [ ]:
# Renombramos la columna de artista del DataFrame
lastfm_id = lastfm_id.rename(columns={'Nombre': 'artista'})

In [ ]:
# Unimos lista_artistas_id con lastfm_id para añadir el id_artista correspondiente
lastfm_id = pd.merge(lista_artistas_id, lastfm_id, on='artista', how='left')

In [ ]:
lastfm_id = lastfm_id.drop_duplicates()

In [ ]:
# Eliminamos la columna artista, por no ser necesaria en la futura tabla
lastfm_id = lastfm_id.drop(columns=['artista'])

In [ ]:
# Creamos el csv que nos servirá para la insercción de datos en SQL
lastfm_id.to_csv('lastfm_id.csv', index=False)

In [99]:
lastfm_id

,id_artista,Biografia,Listeners,Playcount,Artistas similares
0,1,Tory Lanez was born Daystar Peterson on July 2...,1447287.0,87028513.0,"['Bryson Tiller', 'PARTYNEXTDOOR', 'Drake', 'A..."
1,2,NaN,NaN,NaN,NaN
2,3,"Idris Vicuña (born May 28, 1990), professional...",1611130.0,49492074.0,"['Yot Club', 'TV Girl', 'paulie leparik', 'Mac..."
3,4,There are at least 2 artists which use this na...,796769.0,14249901.0,"['TV Girl', 'The Simps', 'Lunar Vacation', 'VA..."
4,5,"Tate Rosner McRae (born July 1, 2003) is a Can...",1420625.0,79341249.0,"['Madison Beer', 'Nessa Barrett', 'Gracie Abra..."
...,...,...,...,...,...
1885,1886,"Tyquian Terrel Bowman (born March 23, 1999), b...",149548.0,5033914.0,"['YoungBoy Never Broke Again', 'NoCap', 'JayDa..."
1886,1887,NaN,111855.0,645822.0,"['TisaKorean', 'Lijay', 'A$AP ANT', 'DaBaby', ..."
1887,1888,NaN,48326.0,867091.0,"['Gawne', 'EKOH', 'Chris Webby', 'Merkules', '..."
1888,1889,"Fat Nick (born September 6, 1994) is a Floridi...",190007.0,3078652.0,"['Shakewell', 'Pouya', 'BOOBIE LOOTAVELI', 'Po..."
